# Getting Flow instances from other CoLink users

This notebook demonstrates how a user can obtain instances of Flows that are served by other CoLink users. Both the client user and the serving user need to have a get_instance worker running to perform the instance exchange. 

When a user calls get_flow_instance on some flow_endpoint, we start a colink task with all users that are serving the subflows of the specified Flow. Every involved user will then recursively call get_flow_instance on its served subflows, which might again propagate across different CoLink users. Therefore, a get_flow_instance call results in a tree-like fetching process across CoLink users - this process is facilitated by the get_instance workers.

In [1]:
%load_ext autoreload
%autoreload 2
import os, json
from colink import CoLink
from aiflows.utils import serving
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread, run_get_instance_worker_thread

/Users/nicolasbaldwin/opt/miniconda3/envs/mockenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create two users

In [2]:
cls = colink_utils.start_colink_server_with_users(num_users=2)
print("User 0 id:", cls[0].get_user_id())
print("User 1 id:", cls[1].get_user_id())

User 0 id: 03b4f8bd347a581727fdb0b31ac81d1a17da46aba1e1a440ceb8c0182829699f76
User 1 id: 03964cddf0fa1ba008d786fc7884c9d179c5a9d6118e4aede0c68539a33e031d6e


# USER 0

User 0 will serve ReverseNumberAtomicFlow that User 1 will then use as a subflow.

In [3]:
run_dispatch_worker_thread(cls[0])
run_get_instance_worker_thread(cls[0])

[2024-04-08 10:32:29,053][aiflows.workers.dispatch_worker:236][INFO] - Dispatch worker started in attached thread.
[2024-04-08 10:32:29,055][aiflows.workers.dispatch_worker:237][INFO] - dispatch_point: coflows_dispatch
[2024-04-08 10:32:29,058][aiflows.workers.get_instance_worker:164][INFO] - get_instances worker started in attached thread for user 03b4f8bd347a581727fdb0b31ac81d1a17da46aba1e1a440ceb8c0182829699f76


In [4]:
serving.serve_flow(
    cl=cls[0],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number_atomic",
    singleton=True,
)

[2024-04-08 10:32:31,295][aiflows.utils.serving:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
[2024-04-08 10:32:31,296][aiflows.utils.serving:117][INFO] - dispatch_point: coflows_dispatch
[2024-04-08 10:32:31,296][aiflows.utils.serving:118][INFO] - parallel_dispatch: False
[2024-04-08 10:32:31,298][aiflows.utils.serving:119][INFO] - singleton: True



True

# USER 1

User 1 will get an instance of a local Composite Flow which has remote subflows (served by User 0).

In [5]:
run_dispatch_worker_thread(cls[1])
run_get_instance_worker_thread(cls[1])

[2024-04-08 10:32:32,933][aiflows.workers.dispatch_worker:236][INFO] - Dispatch worker started in attached thread.
[2024-04-08 10:32:32,941][aiflows.workers.dispatch_worker:237][INFO] - dispatch_point: coflows_dispatch
[2024-04-08 10:32:32,950][aiflows.workers.get_instance_worker:164][INFO] - get_instances worker started in attached thread for user 03964cddf0fa1ba008d786fc7884c9d179c5a9d6118e4aede0c68539a33e031d6e


In [6]:
serving.serve_flow(
    cl=cls[1],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlow",
    flow_endpoint="reverse_number_sequential",
)

[2024-04-08 10:32:35,619][aiflows.utils.serving:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
[2024-04-08 10:32:35,620][aiflows.utils.serving:117][INFO] - dispatch_point: coflows_dispatch
[2024-04-08 10:32:35,622][aiflows.utils.serving:118][INFO] - parallel_dispatch: False
[2024-04-08 10:32:35,625][aiflows.utils.serving:119][INFO] - singleton: False



True

In [7]:
# since we are creating new dummy users on every run, we inject user_id into config manually
# typically this would be hardcoded in yaml config
cfg_overrides = {
    "subflows_config": {
        "first_reverse_flow": {
            "user_id": cls[0].get_user_id()
        },
        "second_reverse_flow": {
            "user_id": cls[0].get_user_id()
        }
    }
}

flow = serving.get_flow_instance(
    cl=cls[1],
    flow_endpoint="reverse_number_sequential",
    user_id="local",
    config_overrides = cfg_overrides,
)

[2024-04-08 10:32:41,648][aiflows.workers.get_instance_worker:112][INFO] - 
~~~ serving get_instances request ~~~
[2024-04-08 10:32:41,650][aiflows.workers.get_instance_worker:113][INFO] - task_id = ad9f9067-c9f5-433e-851d-20de125e1cac
[2024-04-08 10:32:43,767][aiflows.workers.get_instance_worker:117][INFO] - get_instance_calls: [['first_reverse_flow', 'reverse_number_atomic', {'flow_class_name': 'ReverseNumberFlowModule.ReverseNumberAtomicFlow', 'singleton': True, 'name': 'ReverseNumberFirst', 'description': 'A flow that takes in a number and reverses it.'}], ['second_reverse_flow', 'reverse_number_atomic', {'flow_class_name': 'ReverseNumberFlowModule.ReverseNumberAtomicFlow', 'name': 'ReverseNumberSecond', 'description': 'A flow that takes in a number and reverses it AGAIN.'}]]
[2024-04-08 10:32:44,150][aiflows.utils.serving:336][INFO] - Mounted f711b422-7eb5-4c3e-9d3f-1c93a876efbf at flows:reverse_number_atomic:mounts:03964cddf0fa1ba008d786fc7884c9d179c5a9d6118e4aede0c68539a33e031d6

[2024-04-08 10:32:41,653][aiflows.workers.get_instance_worker:68][INFO] - 
~~~ get_instances initiator ~~~
[2024-04-08 10:32:41,657][aiflows.workers.get_instance_worker:69][INFO] - task_id = ad9f9067-c9f5-433e-851d-20de125e1cac
[2024-04-08 10:32:44,394][aiflows.workers.get_instance_worker:93][INFO] - Received subflow instances: {
    "first_reverse_flow": {
        "flow_id": "f711b422-7eb5-4c3e-9d3f-1c93a876efbf",
        "successful": true,
        "message": "Fetched local flow instance."
    },
    "second_reverse_flow": {
        "flow_id": "f711b422-7eb5-4c3e-9d3f-1c93a876efbf",
        "successful": true,
        "message": "Fetched local flow instance."
    }
}
[2024-04-08 10:32:47,405][aiflows.workers.dispatch_worker:119][INFO] - 
~~~ Dispatch task ~~~
[2024-04-08 10:32:47,434][aiflows.workers.dispatch_worker:161][INFO] - flow_endpoint: reverse_number_sequential
[2024-04-08 10:32:47,434][aiflows.workers.dispatch_worker:162][INFO] - flow_id: 25886058-80c2-49ba-8b06-c9102d312667

[2024-04-08 10:32:44,464][aiflows.utils.serving:336][INFO] - Mounted 25886058-80c2-49ba-8b06-c9102d312667 at flows:reverse_number_sequential:mounts:local:25886058-80c2-49ba-8b06-c9102d312667


In [8]:
input_data = {"id": 0, "number": 1234}
    
input_message = flow.package_input_message(input_data)
future = flow.get_reply_future(input_message)
reply_data = future.get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

Data sent:
 {'id': 0, 'number': 1234} 

REPLY:
 {'output_number': 1234} 



#### Directly fetching remote Flows

User 1 can also directly call get_instance on the Flows served by User 0.

In [9]:
remote_flow = serving.get_flow_instance(
    cl=cls[1],
    flow_endpoint="reverse_number_atomic",
    user_id=cls[0].get_user_id(),
) # fetches the singleton served by User 0

In [10]:
input_data = {"id": 0, "number": 1234}
input_message = remote_flow.package_input_message(input_data)
remote_flow.get_reply_future(input_message).get_data()

{'output_number': 4321}

# Observe storage of both users

User 0 has a serve endpoint for ReverseNumberAtomic flow and a singleton instance of that flow.
User 1 has a serve endpoint for ReverseNumberSequential flow and an instance of that flow whose subflows are remote instances of the ReverseNumberAtomic at user 0 (since ReverseNumberAtomic flow is a singleton, both subflows are the same flow instance).

In [11]:
def status(cl):
    colink_utils.print_served_flows(cl, print_values=False)
    print("\nAll flow instances:")
    colink_utils.print_flow_instances(cl)

In [12]:
status(cls[0])

 reverse_number_atomic/
   parallel_dispatch
   default_dispatch_point
   flow_class_name


   singleton
   mounts/
     03964cddf0fa1ba008d786fc7884c9d179c5a9d6118e4aede0c68539a33e031d6e/
       f711b422-7eb5-4c3e-9d3f-1c93a876efbf/
         init
         config_overrides
         state
   init

All flow instances:
 f711b422-7eb5-4c3e-9d3f-1c93a876efbf


In [13]:
status(cls[1])

 reverse_number_sequential/
   init
   flow_class_name
   singleton
   default_dispatch_point
   mounts/
     local/
       25886058-80c2-49ba-8b06-c9102d312667/
         init
         config_overrides
         state
   parallel_dispatch

All flow instances:


 25886058-80c2-49ba-8b06-c9102d312667
